In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
from skimage import io
import math

In [28]:
image = io.imread('../musicalrobot/data/Lepton_Capture.tiff')

In [29]:
print(image.shape)

(196, 120, 160)


In [30]:
image[0]

array([[29993, 29984, 29966, ..., 29912, 29906, 29914],
       [30001, 29976, 29972, ..., 29914, 29923, 29929],
       [29982, 29974, 29982, ..., 29916, 29906, 29933],
       ...,
       [30113, 30062, 30068, ..., 29862, 29891, 29893],
       [30093, 30068, 30081, ..., 29875, 29873, 29902],
       [30087, 30077, 30066, ..., 29877, 29870, 29896]], dtype=uint16)

In [31]:
frames, height, width = image.shape
print(frames)

196


### Import image and read statistics

In [32]:
image = io.imread('../musicalrobot/data/Lepton_Capture.tiff')
frames, height, width = image.shape

### Determining dimensions - find origin

In [33]:
frames, height, width = image.shape
rowmid = int(width / 2)
colmid = int(height / 2)


###  Choosing specific frames

In [34]:
frame = 0
factor = 14 #approx half a second
chosenframes = []

while frame <= frames:
    if frame % factor == 0:
        chosenframes.append(frame)
        frame = frame + 1
    else:
        frame = frame + 1

In [35]:
print(chosenframes)

[0, 14, 28, 42, 56, 70, 84, 98, 112, 126, 140, 154, 168, 182, 196]


In [36]:
chosenframes[3]

42

### Read single temperature in single frame

In [37]:
imarray = np.array( image)
imarray.shape

(196, 120, 160)

In [38]:
image.shape

(196, 120, 160)

Celsius - max - frame 0

In [39]:
framearray = imarray[56]
C = (framearray.max() - 27315) / 100 
C

37.91

Fahrenheit - max - frame 0

In [40]:
framearray = imarray[0]
C = (framearray.max() - 27315) / 100 
F = C * 9 / 5 + 32
F

100.022

In [41]:
def centikelvin_to_celsius(temp):
    cels = (temp - 27315)/100
    return cels

def to_fahrenheit(temp):
    cels = centikelvin_to_celsius(temp)
    fahr = cels * 9 / 5 + 32
    return fahr

def to_temperature(temp):
    cels = centikelvin_to_celsius(temp)
    fahr = cels * 9 / 5 + 32
    return cels, fahr


In [42]:
framearray = imarray[0]
temp = framearray[colmid,rowmid]

cels, fahr = to_temperature(temp)
fahr = to_fahrenheit(temp)

print("celsius: ", cels, " , fahrenheit: ",fahr)

celsius:  36.01  , fahrenheit:  96.818


Possible temperatures:
* Min
* Max
* Mean
* Any predetermined point - center

### Loop through multiple chosen frames with single temperature

In [43]:
maxlength = len(chosenframes)
alltempc = []
alltempf = []
index = 0
print(maxlength)

15


In [46]:
while index < maxlength -1:
    frame = chosenframes[index]
    framearray = imarray[frame]
    temp = framearray[colmid,rowmid]
    cels, fahr = to_temperature(temp)
    alltempc.append(cels)
    alltempf.append(fahr)
    index = index + 1

In [47]:
print(alltempc)

[36.01, 28.06, 35.49, 32.66, 35.92, 35.39, 35.6, 35.81, 35.64, 35.52, 36.3, 29.94, 31.94, 32.05]


### Indexing time over frames for a single temperature
Assumptions: frame per a second is 27 - from Lepton FLIR manual talking about one frame video data being in 1/27th of a second. 

In [48]:
alltime = []
maxlength = len(chosenframes)
index = 0

while index < maxlength:
    frame = chosenframes[index]
    time = frame / 27
    alltime.append(round(time, 2))
    index = index + 1

In [49]:
print(alltime)

[0.0, 0.52, 1.04, 1.56, 2.07, 2.59, 3.11, 3.63, 4.15, 4.67, 5.19, 5.7, 6.22, 6.74, 7.26]


### Create dataframe of all data

In [3]:
names = ['component1', 'component two']

In [4]:
mol_data = pd.DataFrame()

mol_data[names[0]]

KeyError: 'component1'

In [50]:
data = pd.DataFrame()
data['Frame'] = chosenframes
data['Time'] = alltime
data['Temp (C)'] = alltempc
data['Temp (F)'] = alltempf
data

ValueError: Length of values does not match length of index

### Graphing change of temperature over time 

In [51]:
plt.plot(data['Time'], data['Temp (F)'])

KeyError: 'Temp (F)'

### Reading all temperature in single frame

In [52]:
framearray = imarray[0]
alltempall = np.zeros((height, width))
row = 0
col = 0

while col < height :
    row = 0
    while row < width:
        temp = framearray[col, row]
        cels, fahr = to_temperature(temp)
        alltempall[col, row] = cels
        row += 1
    col += 1

In [53]:
print(alltempall)

[[26.78 26.69 26.51 ... 25.97 25.91 25.99]
 [26.86 26.61 26.57 ... 25.99 26.08 26.14]
 [26.67 26.59 26.67 ... 26.01 25.91 26.18]
 ...
 [27.98 27.47 27.53 ... 25.47 25.76 25.78]
 [27.78 27.53 27.66 ... 25.6  25.58 25.87]
 [27.72 27.62 27.51 ... 25.62 25.55 25.81]]
